In [1]:
import json
from nltk.translate import bleu_score
from numpy import dot
from numpy.linalg import norm
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
import torch
import glob


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path,trust_remote_code=True).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/57.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [3]:
similarity_scores = {'llm_without_rag':[],'rag_with_compression':[],'rag_with_rerank_similarity':[],'rag_with_rerank_mmr':[],'rag_with_compression_rerank':[]}
blue_scores =  {'llm_without_rag':[],'rag_with_compression':[],'rag_with_rerank_similarity':[],'rag_with_rerank_mmr':[],'rag_with_compression_rerank':[]}
answers_folder_path = "/content/drive/MyDrive/MS IIITH/MS CourseWork/SMAI/Project/RAG_answers"
for file in glob.glob(answers_folder_path+"/*.json"):
  with open(file, "r") as json_file:
      data = json.load(json_file)
  print(file.split("/")[-1])
  for question in data:
    chatgpt_ans = data[question]['chat_gpt_answer'].strip()
    llm_without_rag = data[question]['llm_without_rag'].replace(question,"").strip()
    rag_with_compression = data[question]['rag_with_compression'].strip()
    rag_with_rerank_similarity = data[question]['rag_with_rerank_similarity'].strip()
    rag_with_rerank_mmr = data[question]['rag_with_rerank_mmr'].strip()
    rag_with_compression_rerank = data[question]['rag_with_compression_rerank'].strip()

    tokenized = tokenizer([chatgpt_ans], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    chatgpt_ans_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    tokenized = tokenizer([llm_without_rag], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    llm_without_rag_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    tokenized = tokenizer([rag_with_compression], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    rag_with_compression_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    tokenized = tokenizer([rag_with_rerank_similarity], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    rag_with_rerank_similarity_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    tokenized = tokenizer([rag_with_rerank_mmr], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    rag_with_rerank_mmr_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    tokenized = tokenizer([rag_with_compression_rerank], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
    rag_with_compression_rerank_e = model(**tokenized).last_hidden_state[:, 0].detach().cpu().numpy()

    similarity_scores['llm_without_rag'].append(dot(chatgpt_ans_e.squeeze(), llm_without_rag_e.squeeze())/(norm(chatgpt_ans_e)*norm(llm_without_rag_e)))
    similarity_scores['rag_with_compression'].append(dot(chatgpt_ans_e.squeeze(), rag_with_compression_e.squeeze())/(norm(chatgpt_ans_e)*norm(rag_with_compression_e)))
    similarity_scores['rag_with_rerank_similarity'].append(dot(chatgpt_ans_e.squeeze(), rag_with_rerank_similarity_e.squeeze())/(norm(chatgpt_ans_e)*norm(rag_with_rerank_similarity_e)))
    similarity_scores['rag_with_rerank_mmr'].append(dot(chatgpt_ans_e.squeeze(), rag_with_rerank_mmr_e.squeeze())/(norm(chatgpt_ans_e)*norm(rag_with_rerank_mmr_e)))
    similarity_scores['rag_with_compression_rerank'].append(dot(chatgpt_ans_e.squeeze(), rag_with_compression_rerank_e.squeeze())/(norm(chatgpt_ans_e)*norm(rag_with_compression_rerank_e)))



    blue_scores['llm_without_rag'].append(bleu_score.sentence_bleu([chatgpt_ans], llm_without_rag))
    blue_scores['rag_with_compression'].append(bleu_score.sentence_bleu([chatgpt_ans], rag_with_compression))
    blue_scores['rag_with_rerank_similarity'].append(bleu_score.sentence_bleu([chatgpt_ans], rag_with_rerank_similarity))
    blue_scores['rag_with_rerank_mmr'].append(bleu_score.sentence_bleu([chatgpt_ans], rag_with_rerank_mmr))
    blue_scores['rag_with_compression_rerank'].append(bleu_score.sentence_bleu([chatgpt_ans], rag_with_compression_rerank))





Transformer_CTC.json
TDNN.json
wav2vec2.json
SpeechBrain.json
Espnet.json
Kaldi.json
LAS.json
Survey.json


In [4]:
for ans_type in similarity_scores:
  score_t = sum(similarity_scores[ans_type])/len(similarity_scores[ans_type])
  print(f"{ans_type}:{score_t}:{len(similarity_scores[ans_type])}")

llm_without_rag:0.788235878944397:80
rag_with_compression:0.8319168567657471:80
rag_with_rerank_similarity:0.8409862704575062:80
rag_with_rerank_mmr:0.8380542568862438:80
rag_with_compression_rerank:0.843903411924839:80


In [5]:
for ans_type in blue_scores:
  score_t = sum(blue_scores[ans_type])/len(blue_scores[ans_type])
  print(f"{ans_type}:{score_t}:{len(similarity_scores[ans_type])}")

llm_without_rag:0.10804516212886998:80
rag_with_compression:0.26626375857188417:80
rag_with_rerank_similarity:0.2491545603051713:80
rag_with_rerank_mmr:0.24790966147397303:80
rag_with_compression_rerank:0.28492950873454037:80
